### Задание "Ставки"

Вы – маркетинговый аналитик в компании Озон.

Когда-то давно, в 2019 году, когда Озон был просто интернет-магазином, Вашей задачей было привлечение клиентов и продвижение товаров интернет-магазина.

Сейчас, когда Озон стал маркетплейсом, дирекция по маркетингу решила продавать свои услуги независимым продавцам. 

Рассматривается следующая схема взаимодействия:
 - Пользователь, зайдя на сайт, делает **поисковый запрос**
 - Поисковому запросу соответствует своя натуральная выдача
 - Выдача есть упорядоченный набор товаров
 - Каждый товар характеризуется релевантностью запросу (**relevant**), вероятностью покупки (**probability**), стоимостью (**price**), а также маржинальностью (**marginality**) (эти данные у нас есть).
 - Каждый клиент характеризуется своими запросами и своим бюджетом (**dict_of_queries**) (для каждого запроса) и набором личных характеристик (**dict_of_characteristics**) (фичей: пол, статус, размер и тд) – эти данные мы можем оценить.
 - Селлер может заплатить Озон за продвижение своего товара в поисковой выдаче. «Ставкой» селлера является дополнительная маржа при продаже (**margin_to_pay_back**), которой он делится с маркетплейсом.
   
**Ваша задача:** постройте **математическую модель** (**def propose_margin_to_pay_back()**), описывающую процесс принятия решений для селлера. Предложите на базе модели **алгоритм** (**def change_rank_of_sellers_good(margin_to_pay_back)**), который получая от селлера размер его ставки определяет, как именно изменить позицию товара селлера в поисковой выдаче. 

**Обоснуйте качество решения алгоритма**.

### Решение

Формализуем задачу

Пусть у нас есть словарь (либо таблица в базе данных) **dict_of_clients** с информацией о 100 пользователях.

In [1]:
dict_of_clients = {f'C{"0" * (2 - len(str(i)))}{i}':{'dict_of_characteristics':{'age':0,
                                                                                'gender':'',
                                                                                'size':''},
                                                     'dict_of_queries':{}} for i in range(100)}

У каждого пользователя есть набор личных характеристик **dict_of_characteristics**

Также у каждого пользователя есть набор запросов **dict_of_queries** с описанием каждого запроса вида **запрос-бюджет**

Создаем:
- лист с 100 значениями возраста из нормального распределения со средним значением в 30 лет и стандартным отклонением 10 лет
- лист с возможными значениями пола (также задаем вероятности для каждого значения из листа)
- лист с возможными вариантами размера (задаем вероятности для каждого значения из листа)

In [2]:
from random import seed, choices, normalvariate, randrange, random

seed(a=222, version=2)

mu = 30.0
sigma = 10.0

age = []
while len(age) != 100:
    age_i = int(round(normalvariate(mu, sigma)))
    if age_i >= 18:
        age.append(age_i)

gender = ['male', 'female', 'unknown']
gender_weights = [0.48, 0.48, 0.04]

size = ['other', 'xs', 's', 'm', 'l', 'xl', 'xxl']
size_weights = [0.13, 0.1, 0.14, 0.16, 0.17, 0.16, 0.14]


Соберем возможные варианты запросов (заодно формируем список продуктов, немного странный правда :))

In [3]:
from requests import get
from bs4 import BeautifulSoup

webpage = get('https://zarabotaydengi.com/10-trendovyh-tovarov-dlya-prodazhi-v-2019-godu/').text
product_names = [tag.get_text() for tag in BeautifulSoup(webpage, 'html.parser').find_all('span')][65:-111]
product_names.remove('Корм \u200b\u200bдля домашних животных')
product_names += ['Корм для домашних животных']

queries = list(set([query.split(' ')[0] for query in product_names] + product_names))


Создаем словарь с данными. Для формирования запросов выбираем максимальное число запросов, равное 10, максимальный бюджет - 50 000 у.е.

In [4]:
for client_idx, client in enumerate(dict_of_clients):
    dict_of_clients[client]['dict_of_characteristics']['age'] = age[client_idx]
    dict_of_clients[client]['dict_of_characteristics']['gender'] = choices(gender, gender_weights, k=1)
    dict_of_clients[client]['dict_of_characteristics']['size'] = choices(size, size_weights, k=1)
    
    for query_i in range(randrange(10)):
        dict_of_clients[client]['dict_of_queries'][query_i] = {'query':choices(queries), 
                                                               'budjet':randrange(50000)}

База товаров будет выглядеть следующим образом.

In [6]:
products = {}
seller = [chr(i) for i in range(65,90)]

In [68]:
for product_id in [f'P{"0" * (6 - len(str(i)))}{i}' for i in range(500)]:
    products[product_id] = {'product_name':choices(product_names)[0],
                            'seller':choices(seller)[0],
                            'probability':random(),
                            'price':randrange(50000),
                            'marginality':normalvariate(0.8, 0.2),
                            'margin_to_pay_back': randrange(3000)}

Поле 'relevant' будет иметь три значения: 1, 5 и 10 и зависит от запроса
- 1 означает, что запрос не содержится в названии продукта
- 5 означает, что запрос совпадает с первым словом названия продукта
- 10 означает, что запрос полностью совпадает с названием продукта

#### Процесс принятия решения для селлера в предположении, что он может установить только единую для каждого случая ставку

Задача селлера - подобрать такую ставку, чтобы общая выручка выросла с ростом объема продаж, обусловленным продвижением своего товара в поисковой выдаче.

Вероятно, селлер не имеет доступа к полной информации о покупателе (о всех его предыдущих запросах), а OZON заинтересован в том, чтобы селлер установил какую-либо ставку.

Крайние случаи для селлера:
- Если товар селлера и так постоянно выдается первым, то он не заинтересован в установлении ставки (однако, селлер не знает, всегда ли его товар попадает первым в выдаче, так как это напрямую зависит от формулировки запроса пользователем)
- Если товар селлера и так имеет минимальную маржинальность, то он также не заинтересован в установлении ставки

Предполагаем, что изначально выдача формируется на основе релевантности запросу.
Это то, на что селлер повлиять не может. Допустим также, что вероятность покупки зависит от стоимости товара, поэтому увеличивать стоимость мы не будем. Соответственно, при установлении ставки дополнительная маржа, которую селлер отдает маркетплейсу, уменьшает маржинальность.

Пусть селлер имеет ориентировочную минимальную маржинальность, которую хочет получить, максимальное количество товара, которое он может продать за единицу времени, а также минимальную надбавку общей возможной маржи при заданном максимальном количестве товара и таком увеличении вероятности продажи товара.

Таким образом, мы можем вычислить максимальную ставку, которую готов заплатить селлер маркетплейсу.
Минимальной ставкой считаем отсутсвие ставки, то есть отсутствие целесообразности для селлера платить за подъем своего товара в выдаче поискового запроса.

На старте для селлера будем считать, что увеличение ставки на n * 1% от начальной маржинальности приводит к увеличению вероятности продажи товара на n * 2% от начальной вероятности.

Значит, если разница между текущей и минимальной маржинальностью меньше 1% от текущей, ставка будет равна 0.

Также можно высчитать минимальное n для указанных условий. Стоит рассмотреть 2 случая.

- 1) Рассчитаем максимальное выгодное n в общем случае, независимо от потенциальных ограничений значений curr_probability (p1) и curr_marginality (m1). Для удобства будет считать, что n выражено в долях единицы. Задача селлера - использовать как можно меньшее выгодное ему n.

Получаем функцию 

$F(n) = (p_1 (1 + 2n))(m_1 (1 - n)) - p_1 m_1 = (p_1 + 2p_1 n))(m_1 - m_1 n)) - p_1 m_1 = (p_1 m_1 - p_1 m_1 n + 2 p_1 m_1 n - 2 p_1 m_1 n^2) - p_1 m_1 =$

$= p1 m1 n - 2 p1 m_1 n^2$

Можно переписать это выражение, выделив квадрат

$= - 2 p_1 m_1 (n - \frac{1}{4})^2 + \frac{p_1 m_1}{8}$

Найдем первую производную и экстремум. Так как ветви параболы будут направлены вниз (знак минус перед $n^2$), то мы найдем максимум функции.

$F'(n) = p_1 m_1 - 4 p_1 m_1 n$

Получим в долях n = 0.25, то есть в процентах **n = 25**.

- 2) Однако мы знаем, что вероятность не может быть больше 1, поэтому данное значение работает для $p_1 + 2 * 0.25 p_1 <= 1, т.е. p_1 <= 2/3$)

Если же вероятность выше, то максимальное n находится между округленным вниз и округленным вверх значением $(\frac{1}{p_1} - 1) / 2$ и зависит в том числе от $m_1$


Если максимальное n, посчитанное исходя из разницы минимальной и текущей маржинальности, больше найденного аналитически n, то принимаем за n найденного аналитически.

Далее сравниваем минимальную надбавку общей возможной маржи с имеющейся разницей. Если при максимально возможном n мы не достигаем минимальной разницы, то ставка будет равна 0. 

In [8]:
from math import ceil, floor
import numpy as np

In [9]:
def calc_delta_marge(product_id, max_product_value, optim_n):
    """
    Функция возвращает разницу между текущим значением общей маржи с учетом продажи max_product_value 
    количества товаров с характеристиками product_id, с учетом оптимального значения коэффициента увеличения 
    ставки optim_n
    """
    curr_probability = products[product_id]['probability']
    price = products[product_id]['price']
    curr_marginality = products[product_id]['marginality']
    
    gross_marge_curr = max_product_value * curr_probability * curr_marginality * price
    
    min_marginality = curr_marginality * (1 - optim_n / 100)
    max_probability = max(1, curr_probability * (1 + 2 * optim_n / 100))
    
    gross_marge_max_margin_to_pay_back = max_product_value * max_probability * min_marginality * price
    
    return gross_marge_max_margin_to_pay_back - gross_marge_curr

def propose_margin_to_pay_back(product_id, max_product_value, min_marginality=0, min_gross_marge_delta=0):
    """
    Функция возвращает оптимальное для селлера значение ставки в у.е. в зависимости от характеристик 
    продукта product_id, максимального количества товаров max_product_value, минимальной необходимой 
    селлеру маржинальности min_marginality и минимальной надбавки общей возможной маржи min_gross_marge_delta
    """
    curr_probability = products[product_id]['probability']
    price = products[product_id]['price']
    curr_marginality = products[product_id]['marginality']
    
    # marge = curr_marginality * price
    # new_marginality = (curr_marginality * price - margin_to_pay_back) / price
    # new_marginality >= min_marginality
    
    delta_curr_min_marginality = curr_marginality - min_marginality
    max_n = int(100 * delta_curr_min_marginality // curr_marginality)  
    min_margin_to_pay_back = 0
    
    if max_n < 1:
        return min_margin_to_pay_back
    
    if curr_probability <= 2/3:
        optim_n1 = min(25, max_n)
        optim_n2 = min(25, max_n)
    else:
        optim_n1 = min(ceil((100 / curr_probability - 100) / 2), max_n)
        optim_n2 = min(floor((100 / curr_probability - 100) / 2), max_n)
    
    arr_of_n = np.array([optim_n1, optim_n2])
    arr_of_deltas = np.vectorize(calc_delta_marge)(product_id, max_product_value, arr_of_n)
    
    if max(arr_of_deltas) < min_gross_marge_delta:
        return min_margin_to_pay_back
    
    n = arr_of_n[np.argmax(arr_of_deltas)]
    margin_to_pay_back = curr_marginality * n * price / 100
    
    return floor(margin_to_pay_back)

##### Например

In [10]:
product_id, max_product_value, min_marginality, min_gross_marge_delta = 'P000000', 10000, 0.6, 10000000
margin_to_pay_back = propose_margin_to_pay_back(product_id, max_product_value, 
                                                min_marginality, min_gross_marge_delta)

In [11]:
print(f"""Для товара с артикулом {product_id}, вероятностью продажи {round(products[product_id]['probability'], 2)}, текущей маржинальностью {round(products[product_id]['marginality'], 2)} и стоимостью {products[product_id]['price']} у.е., 
максимальным количеством товара на складе {max_product_value} единиц, минимальной маржинальностью продавца {min_marginality} и минимальной 
общей маржой {min_gross_marge_delta} у.е. селлер выберет ставку в {margin_to_pay_back} у.е.""")

Для товара с артикулом P000000, вероятностью продажи 0.62, текущей маржинальностью 0.91 и стоимостью 9167 у.е., 
максимальным количеством товара на складе 10000 единиц, минимальной маржинальностью продавца 0.6 и минимальной 
общей маржой 10000000 у.е. селлер выберет ставку в 2075 у.е.


In [12]:
product_id, max_product_value, min_marginality, min_gross_marge_delta = 'P000000', 10000, 0.6, 0
margin_to_pay_back = propose_margin_to_pay_back(product_id, max_product_value, 
                                                min_marginality, min_gross_marge_delta)

In [13]:
print(f"""Для товара с артикулом {product_id}, вероятностью продажи {round(products[product_id]['probability'], 2)}, текущей маржинальностью {round(products[product_id]['marginality'], 2)} и стоимостью {products[product_id]['price']} у.е., 
максимальным количеством товара на складе {max_product_value} единиц, минимальной маржинальностью продавца {min_marginality} и минимальной 
общей маржой {min_gross_marge_delta} у.е. селлер выберет ставку в {margin_to_pay_back} у.е.""")

Для товара с артикулом P000000, вероятностью продажи 0.62, текущей маржинальностью 0.91 и стоимостью 9167 у.е., 
максимальным количеством товара на складе 10000 единиц, минимальной маржинальностью продавца 0.6 и минимальной 
общей маржой 0 у.е. селлер выберет ставку в 2075 у.е.


In [14]:
product_id, max_product_value, min_marginality, min_gross_marge_delta = 'P000001', 10000, 0.6, 0
margin_to_pay_back = propose_margin_to_pay_back(product_id, max_product_value, 
                                                min_marginality, min_gross_marge_delta)

In [15]:
print(f"""Для товара с артикулом {product_id}, вероятностью продажи {round(products[product_id]['probability'], 2)}, текущей маржинальностью {round(products[product_id]['marginality'], 2)} и стоимостью {products[product_id]['price']} у.е., 
максимальным количеством товара на складе {max_product_value} единиц, минимальной маржинальностью продавца {min_marginality} и минимальной 
общей маржой {min_gross_marge_delta} у.е. селлер выберет ставку в {margin_to_pay_back} у.е.""")

Для товара с артикулом P000001, вероятностью продажи 0.25, текущей маржинальностью 1.43 и стоимостью 14940 у.е., 
максимальным количеством товара на складе 10000 единиц, минимальной маржинальностью продавца 0.6 и минимальной 
общей маржой 0 у.е. селлер выберет ставку в 5356 у.е.




Если селлер уже имеет более точную статистику по соотношению "увеличение ставки на 1% от начальной маржинальности / увеличение вероятности продажи товара на n% от начальной вероятности", то ему необходимо будет пересчитать некоторые коэффициенты, начиная с аналитического определения n

In [69]:
import pandas as pd

df_products = pd.DataFrame(products).T

In [70]:
df_products['first_word'] = df_products.product_name.str.split()

In [71]:
df_products['first_word'] = df_products.product_name.str.partition()

In [80]:
df_products['E_margin_to_pay_back'] = df_products.probability * df_products.margin_to_pay_back

In [73]:
df_products.sort_values('E_margin_to_pay_back', inplace=True, ascending=False)

In [74]:
df_products

,product_name,seller,probability,price,marginality,margin_to_pay_back,first_word,E_margin_to_pay_back
P000262,Сетчатая обувь,E,0.999255,27740,0.907189,2572,Сетчатая,2570.08
P000378,Товары для домашних животных,R,0.998137,13736,0.460028,2335,Товары,2330.65
P000217,Чехлы для телефонов,T,0.997953,48163,0.781509,858,Чехлы,856.244
P000454,Корм для домашних животных,J,0.997633,3869,1.0877,94,Корм,93.7775
P000170,Спорт и путешествия,V,0.995401,43221,0.771247,2775,Спорт,2762.24
...,...,...,...,...,...,...,...,...
P000050,Наращивание ногтей,G,0.0159477,42148,0.794618,1881,Наращивание,29.9977
P000424,Умные часы,P,0.0119823,48822,0.826149,1304,Умные,15.625
P000398,Автомобильные чехлы,B,0.00965672,38993,0.852398,53,Автомобильные,0.511806
P000249,Товары для красоты и здоровья,L,0.00834408,7460,0.606121,2064,Товары,17.2222


#### Процесс принятия решения для маркетплейса по тому, как именно изменить позицию товара селлера в поисковой выдаче

Допустим, что на любой запрос пользователя у нас есть в определенной степени релевантный товар.

Для начала опишем процесс стандартной выдачи: сначала идут продукты с релевантностью 10, затем 5, и в конце 1.

Переназначаем для продукта все поля (изменятся ставка и матожидание суммы, получаемой маркетплейсом за продажу продукта).

Для пустого запроса продукты отсортированы по матожиданию суммы, получаемой маркетплейсом за продажу продукта.

Считаем средний бюджет клиента по предыдущим запросам и фильтруем товары по значению, не превышающему его на 20%.

Задача для продукта - попасть на первую страницу выдачи. На ней выводятся первые 10 продуктов.

Составляем лист из значений согласно релевантности и если товар попал в выборку, то возвращаем его позицию в выборке. Если не попал, то возвращаем первое после отфильтрованных по бюджету значение.

In [93]:
def rank_of_sellers_good(product_id_curr, df_products, client_id, query_='', margin_to_pay_back=0):
    """
    Функция возвращает позицию, на которой будет расположен товар product_id_curr селлера в поисковой выдаче
    по запросу query_ для пользователя client_id, на основе данных о размере ставки селлера margin_to_pay_back
    
    """
    
    marginality_curr = products[product_id_curr]['marginality']
    probability_curr = products[product_id_curr]['probability']
    price_curr = products[product_id_curr]['price']
    seller = products[product_id_curr]['seller']
    product_name = products[product_id_curr]['product_name']
    E_margin_to_pay_back_curr = probability_curr * margin_to_pay_back
    
    mean_budjet = pd.DataFrame(dict_of_clients[client_id]['dict_of_queries']).T.budjet.mean()
    n_queries = max(dict_of_clients[client_id]['dict_of_queries'])
    
    df_products = df_products[df_products.index != product_id_curr].copy()
    
    df_products.loc[product_id_curr] = [product_name, seller, probability_curr, price_curr, marginality_curr,
                                        margin_to_pay_back, product_name.split(' ')[0], E_margin_to_pay_back_curr]
    
    df_filtered = df_products[df_products.price <= mean_budjet * 1.2].sort_values('E_margin_to_pay_back', 
                                                                                  ascending=False).copy()
    list_query = list(df_filtered[df_filtered.product_name == query_].index)
    list_query += list(df_filtered[df_filtered.first_word == query_].index)
    list_query += list(df_filtered.query('product_name!="query_" and first_word!="query_"').index)
    
    
    
    if product_id_curr in list_query:
        index_ = list_query.index(product_id_curr)
        return index_
        
    return len(df_filtered)
    
    

In [94]:
rank_of_sellers_good('P000454', df_products, 'C05', query_='', margin_to_pay_back=0.6)

324